In [ ]:
import feather
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
train = feather.read_dataframe('../data/interim/train.ftr')
test = feather.read_dataframe('../data/interim/test.ftr')
train.shape, test.shape

((903653, 55), (804684, 53))

In [3]:
geoNetwork_columns = [col for col in train.columns if "geoNetwork" in col]
train[geoNetwork_columns].dtypes

geoNetwork.city               object
geoNetwork.cityId             object
geoNetwork.continent          object
geoNetwork.country            object
geoNetwork.latitude           object
geoNetwork.longitude          object
geoNetwork.metro              object
geoNetwork.networkDomain      object
geoNetwork.networkLocation    object
geoNetwork.region             object
geoNetwork.subContinent       object
dtype: object

### 欠損値

- 欠損はどのカラムにも存在しない。

In [5]:
def find_missing(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return df.loc[~(df['Total']==0)]

miss_train = find_missing(train[geoNetwork_columns])
miss_test = find_missing(test[geoNetwork_columns])
display(miss_train)
display(miss_test)

,Total,Percent


,Total,Percent


### 各カラムの中身を簡易集計

In [6]:
def aggregates_train_test(colname):
    summary_train = train.groupby(colname).count()['date'].sort_values(ascending=False)
    summary_test = test.groupby(colname).count()['date'].sort_values(ascending=False)
    summary_train.name = 'train'
    summary_test.name = 'test'
    result = pd.concat([summary_train, summary_test], axis=1, sort=True, join='outer').sort_values('train', ascending=False)
    return result

In [7]:
for col in geoNetwork_columns_train:
    print(col)
    display(aggregates_train_test(col))

geoNetwork.city


,train,test
not available in demo dataset,508229.0,424730.0
Mountain View,40884.0,33226.0
(not set),34262.0,31605.0
New York,26371.0,23089.0
San Francisco,20329.0,16631.0
Sunnyvale,13086.0,14837.0
London,12607.0,11015.0
San Jose,10295.0,9846.0
Los Angeles,8670.0,8368.0
Bangkok,7709.0,4759.0


geoNetwork.cityId


,train,test
geoNetwork.cityId,,
not available in demo dataset,903653,804684


geoNetwork.continent


...

,train,test
geoNetwork.networkLocation,,
not available in demo dataset,903653,804684


geoNetwork.region


,train,test
not available in demo dataset,508229.0,424730.0
California,107495.0,99174.0
(not set),27827.0,21947.0
New York,26433.0,23300.0
England,13198.0,12626.0
Texas,8749.0,11731.0
Bangkok,7709.0,4759.0
Washington,7642.0,7730.0
Illinois,7585.0,7767.0
Ho Chi Minh,7250.0,NaN


geoNetwork.subContinent


,train,test
Northern America,390657,377688
Southeast Asia,77800,43834
Southern Asia,59321,61741
Western Europe,59114,56039
Northern Europe,58168,53525
Eastern Asia,46919,44153
Eastern Europe,45249,28758
South America,41731,33381
Western Asia,38443,22523
Southern Europe,35780,31404


### geoNetwork.cityについて

- ユーザーの市区町村。ユーザーの IP アドレスまたは地域 ID から取得されます。

In [8]:
train['geoNetwork.city'].nunique(), test['geoNetwork.city'].nunique()

(649, 732)

- not available in demo datasetの割合は50%ほど。結構多い。

In [15]:
len(train[train['geoNetwork.city']=='not available in demo dataset']) / len(train),\
len(test[test['geoNetwork.city']=='not available in demo dataset']) / len(test)

(0.5624161044117598, 0.5278221015951603)

In [130]:
len(train[train['geoNetwork.city']=='(not set)']) / len(train),\
len(test[test['geoNetwork.city']=='(not set)']) / len(test)

(0.037914996132364966, 0.039276287337638126)

### geoNetwork.cityIdについて

- not available in demo datasetしか値が入っていない。

### geoNetwork.continentについて

- IP アドレスに基づいて特定されたセッションの起点となる大陸。
- 入力されていないもの（not set）も僅かに存在する。

In [16]:
train['geoNetwork.continent'].nunique(), test['geoNetwork.continent'].nunique()

(6, 6)

In [17]:
train['geoNetwork.continent'].unique(), test['geoNetwork.continent'].unique()

(array(['Asia', 'Oceania', 'Europe', 'Americas', 'Africa', '(not set)'],
       dtype=object),
 array(['Asia', 'Europe', 'Americas', 'Africa', 'Oceania', '(not set)'],
       dtype=object))

In [20]:
len(train[train['geoNetwork.continent']=='(not set)']), len(test[test['geoNetwork.continent']=='(not set)'])

(1468, 1049)

### geoNetwork.subContinentについて

- ユーザーの IP アドレスに基づいて特定されたセッションの起点となる亜大陸。


In [38]:
train['geoNetwork.subContinent'].nunique(), test['geoNetwork.subContinent'].nunique()

(23, 23)

In [39]:
train['geoNetwork.subContinent'].unique(), test['geoNetwork.subContinent'].unique()

(array(['Western Asia', 'Australasia', 'Southern Europe', 'Southeast Asia',
        'Northern Europe', 'Southern Asia', 'Western Europe',
        'South America', 'Eastern Asia', 'Eastern Europe',
        'Northern America', 'Western Africa', 'Central America',
        'Eastern Africa', '(not set)', 'Caribbean', 'Southern Africa',
        'Northern Africa', 'Central Asia', 'Middle Africa', 'Melanesia',
        'Micronesian Region', 'Polynesia'], dtype=object),
 array(['Southeast Asia', 'Southern Europe', 'Western Europe',
        'Northern America', 'Central America', 'South America',
        'Northern Europe', 'Eastern Asia', 'Western Asia',
        'Eastern Europe', 'Southern Asia', 'Northern Africa',
        'Eastern Africa', 'Western Africa', 'Australasia', 'Caribbean',
        'Central Asia', '(not set)', 'Southern Africa', 'Middle Africa',
        'Micronesian Region', 'Polynesia', 'Melanesia'], dtype=object))

In [40]:
display(train[train['geoNetwork.continent']=='(not set)'].groupby('geoNetwork.subContinent').count()['date'])
display(test[test['geoNetwork.continent']=='(not set)'].groupby('geoNetwork.subContinent').count()['date'])

geoNetwork.subContinent
(not set)    1468
Name: date, dtype: int64

geoNetwork.subContinent
(not set)    1049
Name: date, dtype: int64

### geoNetwork.countryについて

- IP アドレスに基づいて特定されたセッションの起点となる国。


In [29]:
train['geoNetwork.country'].nunique(), test['geoNetwork.country'].nunique()

(222, 219)

### geoNetwork.latitudeについて

- not available in demo datasetしか値が入っていない。

### geoNetwork.longitudeについて

- not available in demo datasetしか値が入っていない。

### geoNetwork.metroについて

- セッションの起点となる指定マーケット エリア（DMA）。

In [34]:
train['geoNetwork.metro'].nunique(), test['geoNetwork.metro'].nunique()

(94, 109)

### geoNetwork.networkDomainについて

- ユーザーのインターネット サービス プロバイダのドメイン名。インターネット サービス プロバイダの IP アドレスに登録されているドメイン名から取得されます。

In [37]:
train['geoNetwork.networkDomain'].nunique(), test['geoNetwork.networkDomain'].nunique()

(28064, 25750)

- unknown.unknownやnot set, voxility.comなどはスパムを除外するのに使われているみたい。
https://analyze.siraberu.info/post-1007/

In [157]:
train.groupby('geoNetwork.networkDomain').size().sort_values(ascending=False).head()

geoNetwork.networkDomain
(not set)          244881
unknown.unknown    146034
comcast.net         28743
rr.com              14827
verizon.net         13637
dtype: int64

- 上のドメインが、本当にスパムっぽいかどうか調べる

In [159]:
# revenueがnullであるなら1, そうでないなら0
train['revenue_flg'] = (train['totals.transactionRevenue']!=train['totals.transactionRevenue']).astype(int)
train.groupby('revenue_flg').size().sort_values(ascending=False).head()

revenue_flg
1    892138
0     11515
dtype: int64

In [164]:
result_grp = train.groupby(['geoNetwork.networkDomain', 'revenue_flg']).size().reset_index()
result_grp[result_grp['geoNetwork.networkDomain'].isin(["unknown.unknown", "(not set)", "voxility.com"])]

,geoNetwork.networkDomain,revenue_flg,0
0,(not set),0,6645
1,(not set),1,238236
26448,unknown.unknown,0,400
26449,unknown.unknown,1,145634
27399,voxility.com,0,1
27400,voxility.com,1,5


### geoNetwork.networkLocationについて

- not available in demo datasetしか値が入っていない。

### geoNetwork.regionについて

- IP アドレスから取得したセッションの起点となる地域。米国では地域はニューヨークなどの州です。


In [43]:
train['geoNetwork.region'].nunique(), test['geoNetwork.region'].nunique()

(376, 376)

In [45]:
len(train[train['geoNetwork.region']=='(not set)']), len(test[test['geoNetwork.region']=='(not set)'])

(27827, 21947)

In [46]:
len(train[train['geoNetwork.region']=='not available in demo dataset']), len(test[test['geoNetwork.region']=='not available in demo dataset'])

(508229, 424730)

### geoNetworkデータの大小関係について

continent(大陸) > subcontinent(亜大陸) > country(国) > region(地域) > city(市区町村)

In [83]:
total = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)
train_index = range(0, len(train))
test_index = range(len(train), len(train)+len(test))
total.shape

(1708337, 55)

In [88]:
total['continent_flg'] = (total['geoNetwork.continent']=='(not set)').astype(int)
total['subcontinent_flg'] = (total['geoNetwork.subContinent']=='(not set)').astype(int)
total['country_flg'] = (total['geoNetwork.country']=='(not set)').astype(int)
total['region_flg'] = (total['geoNetwork.region']=='(not set)').astype(int) + (total['geoNetwork.region']=='not available in demo dataset').astype(int)
total['city_flg'] = (total['geoNetwork.city']=='(not set)').astype(int) + (total['geoNetwork.city']=='not available in demo dataset').astype(int)

- 1は未記入、0は記入

In [89]:
total.groupby(['continent_flg', 'subcontinent_flg', 'country_flg']).count()['date']

continent_flg  subcontinent_flg  country_flg
0              0                 0              1705820
1              1                 1                 2517
Name: date, dtype: int64

In [90]:
total.groupby(['continent_flg', 'subcontinent_flg', 'country_flg', 'region_flg']).count()['date']

continent_flg  subcontinent_flg  country_flg  region_flg
0              0                 0            0             725565
                                              1             980255
1              1                 1            0                 39
                                              1               2478
Name: date, dtype: int64

In [91]:
total.groupby(['continent_flg', 'subcontinent_flg', 'country_flg', 'region_flg', 'city_flg']).count()['date']

continent_flg  subcontinent_flg  country_flg  region_flg  city_flg
0              0                 0            0           0           694504
                                                          1            31061
                                              1           0            14969
                                                          1           965286
1              1                 1            0           0               35
                                                          1                4
                                              1           0                3
                                                          1             2475
Name: date, dtype: int64

- trainsactionRevenueとの相関

In [143]:
train_tmp = total.loc[train_index]

In [144]:
train_tmp['revenue_flg'] = (train_tmp['totals.transactionRevenue']!=train_tmp['totals.transactionRevenue']).astype(int)

In [147]:
train_tmp[train_tmp['geoNetwork.networkDomain']=='unknown.unknown'].groupby('revenue_flg').size()

revenue_flg
0       400
1    145634
dtype: int64

In [148]:
train_tmp[train_tmp['geoNetwork.networkDomain']=='(not set)'].groupby('revenue_flg').size()

revenue_flg
0      6645
1    238236
dtype: int64

In [ ]:
train_tmp[train_tmp['geoNetwork.networkDomain']=='voxility.com'].groupby('revenue_flg').size()

In [103]:
train_tmp.groupby(['continent_flg', 'subcontinent_flg', 'country_flg', 'region_flg', 'city_flg', 'revenue_flg']).count()['date']

continent_flg  subcontinent_flg  country_flg  region_flg  city_flg  revenue_flg
0              0                 0            0           0         0                6875
                                                                    1              345827
                                                          1         0                  14
                                                                    1               14874
                                              1           0         0                  11
                                                                    1                8441
                                                          1         0                4609
                                                                    1              521534
1              1                 1            0           0         0                   1
                                                                    1                   5
                    

### geoNetworkのコンフリクトを解決する

In [354]:
total = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)
train_index = range(0, len(train))
test_index = range(len(train), len(train)+len(test))
total.shape

(1708337, 56)

In [355]:
geoNetwork_columns = [col for col in train.columns if "geoNetwork" in col]
drop_columns = ['geoNetwork.latitude',
                'geoNetwork.longitude',
                'geoNetwork.cityId',
                'geoNetwork.networkLocation',
                'geoNetwork.networkDomain']
geoNetwork_columns = [col for col in geoNetwork_columns if col not in drop_columns]
total[geoNetwork_columns].dtypes

geoNetwork.city            object
geoNetwork.continent       object
geoNetwork.country         object
geoNetwork.metro           object
geoNetwork.region          object
geoNetwork.subContinent    object
dtype: object

In [356]:
for col in geoNetwork_columns:
    total[col] = total[col].where(total[col]!='not available in demo dataset', np.nan)
    total[col] = total[col].where(total[col]!='(not set)', np.nan)
    total[col] = total[col].astype("category").cat.add_categories('N/A').fillna('N/A')

- N/Aの数は？

In [357]:
for col in geoNetwork_columns:
    print(col, '. number of N/A:', (total[col]=='N/A').sum(), 'of the set:', len(total[col]))

geoNetwork.city . number of N/A: 998826 of the set: 1708337
geoNetwork.continent . number of N/A: 2517 of the set: 1708337
geoNetwork.country . number of N/A: 2517 of the set: 1708337
geoNetwork.metro . number of N/A: 1319855 of the set: 1708337
geoNetwork.region . number of N/A: 982733 of the set: 1708337
geoNetwork.subContinent . number of N/A: 2517 of the set: 1708337


#### (continent, subContinent, country)の組み合わせを抽出。
- この組み合わせでは、重複は存在しない。
- 未記入のレコードが2517件存在する。

In [361]:
country_part = total.groupby(
    ['geoNetwork.continent', 'geoNetwork.country',
     'geoNetwork.subContinent']).size().reset_index( )

country_part.head(10)

,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,0
0,Africa,Algeria,Northern Africa,3204
1,Africa,Angola,Middle Africa,118
2,Africa,Benin,Western Africa,135
3,Africa,Botswana,Southern Africa,40
4,Africa,Burkina Faso,Western Africa,153
5,Africa,Burundi,Eastern Africa,58
6,Africa,Cameroon,Middle Africa,414
7,Africa,Cape Verde,Western Africa,31
8,Africa,Central African Republic,Middle Africa,6
9,Africa,Chad,Middle Africa,33


In [362]:
# 重複があるか調べる
country_part[country_part['geoNetwork.country'].duplicated(keep=False)]

,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,0


In [363]:
display(country_part[country_part['geoNetwork.continent']=='N/A'])
display(country_part[country_part['geoNetwork.subContinent']=='N/A'])
display(country_part[country_part['geoNetwork.country']=='N/A'])

,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,0
227,N/A,N/A,N/A,2517


,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,0
227,N/A,N/A,N/A,2517


,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,0
227,N/A,N/A,N/A,2517


- N/Aとなるcountryは、他のカラムを使って補完する。
- city, region, metroだけでは、2517件中2475件の補完ができない。

In [364]:
total[total['geoNetwork.country']=='N/A'].groupby(['geoNetwork.region', 'geoNetwork.city', 'geoNetwork.metro']).size()

geoNetwork.region  geoNetwork.city  geoNetwork.metro                 
Buenos Aires       Buenos Aires     N/A                                     1
California         San Francisco    San Francisco-Oakland-San Jose CA       1
                   Santa Clara      San Francisco-Oakland-San Jose CA       1
County Dublin      Dublin           N/A                                     2
Delhi              N/A              N/A                                     1
England            London           London                                  4
Karnataka          Bengaluru        N/A                                     1
Kyiv city          Kyiv             N/A                                    14
Metro Manila       Manila           N/A                                     1
Minnesota          Minneapolis      Minneapolis-St. Paul MN                 1
Moscow             Moscow           N/A                                     1
New York           New York         New York NY                         

- networkDomainを使えば、2475件中(2475-676)件は補完できる（？）今後の課題

In [365]:
total[(total['geoNetwork.country']=='N/A') &
      (total['geoNetwork.region']=='N/A') &
      (total['geoNetwork.city']=='N/A') &
      (total['geoNetwork.metro']=='N/A')].groupby(['geoNetwork.networkDomain']).size().head()

geoNetwork.networkDomain
(not set)        676
100tb.com         24
16clouds.com       2
alter.net          1
amazonaws.com     53
dtype: int64

- 上記の通り、cityとregionを使って、countryのNULLを埋める。

In [366]:
result = total[total['geoNetwork.country']=='N/A'].groupby(['geoNetwork.region', 'geoNetwork.city']).size()
result

geoNetwork.region  geoNetwork.city
Buenos Aires       Buenos Aires          1
California         San Francisco         1
                   Santa Clara           1
County Dublin      Dublin                2
Delhi              N/A                   1
England            London                4
Karnataka          Bengaluru             1
Kyiv city          Kyiv                 14
Metro Manila       Manila                1
Minnesota          Minneapolis           1
Moscow             Moscow                1
New York           New York              5
Osaka Prefecture   Osaka                 1
Taipei City        N/A                   1
Tamil Nadu         N/A                   2
Victoria           Melbourne             1
Zhejiang           Ningbo                1
N/A                Hong Kong             1
                   Nairobi               1
                   Tunis                 1
                   N/A                2475
dtype: int64

- cityとregionの組み合わせで、最も数の多いcountryを採用すればよさそう。

In [367]:
for idx in result.index:
    region = idx[0]
    city = idx[1]
    
    if region == 'N/A':
        continue
    
    target_index = total[(total['geoNetwork.region']==region) & (total['geoNetwork.city']==city)].index
    data_extr = total.loc[target_index].groupby('geoNetwork.country').size()
    common_country = data_extr.index[data_extr.values.argmax()]
    print('region:', region, ', city:', city, ', selected country:', common_country)
    display(data_extr.sort_values(ascending=False).head(3))
    
    # replace
    total.loc[target_index, 'geoNetwork.country'] = common_country

region: Buenos Aires , city: Buenos Aires , selected country: Argentina


geoNetwork.country
Argentina        3172
United States       7
N/A                 1
dtype: int64

region: California , city: San Francisco , selected country: United States


geoNetwork.country
United States     36699
Canada               33
United Kingdom       31
dtype: int64

region: California , city: Santa Clara , selected country: United States


...

geoNetwork.country
India            847
United States      4
Saudi Arabia       3
dtype: int64

region: Victoria , city: Melbourne , selected country: Australia


geoNetwork.country
Australia        5438
United States      18
Iran                3
dtype: int64

region: Zhejiang , city: Ningbo , selected country: China


geoNetwork.country
China             189
Japan               2
United Kingdom      2
dtype: int64

In [368]:
total[total['geoNetwork.country']=='N/A'].groupby(['geoNetwork.region', 'geoNetwork.city']).size()

geoNetwork.region  geoNetwork.city
N/A                Hong Kong             1
                   Nairobi               1
                   Tunis                 1
                   N/A                2475
dtype: int64

#### (city, region)の組み合わせを抽出。

In [369]:
city_part = total.groupby(['geoNetwork.city', 'geoNetwork.region']).size().reset_index()

city_part.head(10)

,geoNetwork.city,geoNetwork.region,0
0,'s Hertogenbosch,North Brabant,7
1,Aachen,North Rhine-Westphalia,9
2,Aalborg,North Denmark Region,71
3,Aalen,Baden-Wurttemberg,9
4,Aberdeen,Scotland,21
5,Abu Dhabi,Abu Dhabi,105
6,Accra,Greater Accra Region,9
7,Adana,Adana,265
8,Addis Ababa,N/A,104
9,Adelaide,South Australia,44


- cityに値が入っているのに、regionではNULLとなるケースも存在する。
- cityがnairobiの時、countryは｛Kenya, Iceland｝の2種類。まあKenya。
- 基本的には、regionがN/Aのcityはcountryが一意に決まる。
- これは数が少ないので、手動で頑張る。

In [370]:
result = city_part[(city_part['geoNetwork.city'].duplicated(keep=False)) & (city_part['geoNetwork.region'] == 'N/A')]
result

,geoNetwork.city,geoNetwork.region,0
168,Casablanca,N/A,32
209,Colombo,N/A,68
253,Doha,N/A,12
345,Guatemala City,N/A,8
362,Hanoi,N/A,8
550,Minsk,N/A,14
585,Nairobi,N/A,131
883,Tbilisi,N/A,13
1066,N/A,N/A,967761


In [371]:
for idx, row in result.iterrows():
    city = row['geoNetwork.city']
    region = row['geoNetwork.region']
    
    if city == 'N/A':
        continue
    
    target_index = total[(total['geoNetwork.city']==city) & (total['geoNetwork.region']==region)].index
    data_extr = total.loc[target_index].groupby('geoNetwork.country').size()
    common_country = data_extr.index[data_extr.values.argmax()]
    print('region:', region, ', city:', city, ', country:', common_country)
    display(data_extr.sort_values(ascending=False).head(3))

region: N/A , city: Casablanca , country: Morocco


geoNetwork.country
Morocco    32
N/A         0
Gabon       0
dtype: int64

region: N/A , city: Colombo , country: Sri Lanka


geoNetwork.country
Sri Lanka    68
N/A           0
Guernsey      0
dtype: int64

region: N/A , city: Doha , country: Qatar


...

geoNetwork.country
Belarus    14
N/A         0
Gambia      0
dtype: int64

region: N/A , city: Nairobi , country: Kenya


geoNetwork.country
Kenya      129
N/A          1
Iceland      1
dtype: int64

region: N/A , city: Tbilisi , country: Georgia


geoNetwork.country
Georgia    13
N/A         0
Guinea      0
dtype: int64

- regionの候補を決める。
- 一番多いケースを採用。

In [372]:
for idx, row in result.iterrows():
    city = row['geoNetwork.city']
    region = row['geoNetwork.region']
    
    if city == 'N/A':
        continue
    
    target_index = total[(total['geoNetwork.city']==city)].index
    data_extr = total.loc[target_index].groupby('geoNetwork.region').size()
    print(city)
    display(data_extr.sort_values(ascending=False).head(3))

Casablanca


geoNetwork.region
Grand Casablanca     289
N/A                   32
Djak Lak Province      0
dtype: int64

Colombo


geoNetwork.region
Western Province    727
N/A                  68
Hai Duong             0
dtype: int64

Doha


...

geoNetwork.region
Minsk Region        71
N/A                 14
Gia Lai Province     0
dtype: int64

Nairobi


geoNetwork.region
N/A                 131
Nairobi County       74
Gia Lai Province      0
dtype: int64

Tbilisi


geoNetwork.region
Tbilisi      220
N/A           13
Hai Duong      0
dtype: int64

In [373]:
pairs = [('Casablanca', 'Grand Casablanca'), ('Colombo', 'Western Province'), ('Doha', 'Doha'),
         ('Guatemala City', 'Guatemala Department'), ('Hanoi', 'Hanoi'),
         ('Minsk', 'Minsk Region'), ('Nairobi', 'Nairobi County'), ('Tbilisi', 'Tbilisi')]

for c, r in pairs:
    total.loc[(total['geoNetwork.city'] == c) &
              (total['geoNetwork.region'] == 'N/A'), 'geoNetwork.region'] = r

In [374]:
city_part = total.groupby(
    ['geoNetwork.city', 'geoNetwork.region']).size().reset_index()
city_part[(city_part['geoNetwork.city'].duplicated(keep=False)) & (city_part['geoNetwork.region'] == 'N/A')]

,geoNetwork.city,geoNetwork.region,0
1058,N/A,N/A,967761


#### ここまでで、欠損に対する値の補完を行った。次はコンフリクトをチェックする。

- 以下の例のように、同じ(city, region)のペアで様々なcountryが入力されている。
    - cityがアムステルダムで、regionが北ホラント州なのは、オランダ（Netherlands）である。
    - 間違ったcountryが入力されている。

In [376]:
selected = total.loc[:, total.columns.isin(geoNetwork_columns)].copy()
cc = selected.groupby(['geoNetwork.city', 'geoNetwork.region','geoNetwork.country']).size().reset_index()

cc.loc[cc.duplicated(subset=['geoNetwork.city', 'geoNetwork.region'], keep=False)].head(10)

,geoNetwork.city,geoNetwork.region,geoNetwork.country,0
5,Abu Dhabi,Abu Dhabi,United Arab Emirates,104
6,Abu Dhabi,Abu Dhabi,United States,1
13,Ahmedabad,Gujarat,India,2480
14,Ahmedabad,Gujarat,United Kingdom,1
32,Amsterdam,North Holland,Denmark,1
33,Amsterdam,North Holland,Germany,2
34,Amsterdam,North Holland,Indonesia,1
35,Amsterdam,North Holland,Ireland,2
36,Amsterdam,North Holland,Kenya,1
37,Amsterdam,North Holland,Netherlands,3434


- regionしか値が入っていない場合
- regionだけで決めつけるのは早計だと思うが、それでも怪しいcountryが存在する。

In [377]:
result = cc.loc[(cc['geoNetwork.region']!='N/A') & (cc['geoNetwork.city']=='N/A')]
result[result['geoNetwork.region'].duplicated(keep=False)]

,geoNetwork.city,geoNetwork.region,geoNetwork.country,0
1616,N/A,Baja California,Mexico,172
1617,N/A,Baja California,United States,5
1651,N/A,Nevada,Brazil,1
1652,N/A,Nevada,Canada,1
1653,N/A,Nevada,Japan,1
1654,N/A,Nevada,Mexico,2
1655,N/A,Nevada,United Kingdom,6
1656,N/A,Nevada,United States,1394
1657,N/A,New Taipei City,Hong Kong,10
1658,N/A,New Taipei City,Singapore,2


- cityしか値が入っていない場合
- これも怪しい。

In [378]:
result = cc.loc[(cc['geoNetwork.region']=='N/A') & (cc['geoNetwork.city']!='N/A')]
result[result['geoNetwork.city'].duplicated(keep=False)]

,geoNetwork.city,geoNetwork.region,geoNetwork.country,0
532,Hong Kong,N/A,China,2
533,Hong Kong,N/A,Hong Kong,6232
534,Hong Kong,N/A,Indonesia,1
535,Hong Kong,N/A,Japan,6
536,Hong Kong,N/A,Singapore,1
537,Hong Kong,N/A,Taiwan,11
538,Hong Kong,N/A,Thailand,1
539,Hong Kong,N/A,United Kingdom,1
540,Hong Kong,N/A,United States,22
541,Hong Kong,N/A,N/A,1


- 最も入力されているもので代入することで、countryの修正を行う。

In [379]:
most_common = total.groupby([
    'geoNetwork.city', 'geoNetwork.region'
])['geoNetwork.country'].apply(lambda x: x.mode()).reset_index()
most_common.head()

,geoNetwork.city,geoNetwork.region,level_2,geoNetwork.country
0,'s Hertogenbosch,North Brabant,0,Netherlands
1,Aachen,North Rhine-Westphalia,0,Germany
2,Aalborg,North Denmark Region,0,Denmark
3,Aalen,Baden-Wurttemberg,0,Germany
4,Aberdeen,Scotland,0,United Kingdom


In [380]:
for idx, row in most_common.iterrows():
    total.loc[(total['geoNetwork.city'] == row['geoNetwork.city']) &
            (total['geoNetwork.region'] == row['geoNetwork.region']) &
            ((total['geoNetwork.city'] != 'N/A') |
             ((total['geoNetwork.region'] != 'N/A'))
             ), 'geoNetwork.country'] = row['geoNetwork.country']

- 確認

In [381]:
selected = total.loc[:, total.columns.isin(geoNetwork_columns)].copy()
cc = selected.groupby(['geoNetwork.city', 'geoNetwork.region','geoNetwork.country']).size().reset_index()

cc.loc[cc.duplicated(subset=['geoNetwork.city', 'geoNetwork.region'], keep=False)].head(10)

,geoNetwork.city,geoNetwork.region,geoNetwork.country,0
1058,N/A,N/A,Afghanistan,133
1059,N/A,N/A,Albania,737
1060,N/A,N/A,Algeria,3152
1061,N/A,N/A,American Samoa,2
1062,N/A,N/A,Andorra,43
1063,N/A,N/A,Angola,118
1064,N/A,N/A,Anguilla,1
1065,N/A,N/A,Antigua & Barbuda,14
1066,N/A,N/A,Argentina,6373
1067,N/A,N/A,Armenia,442


In [382]:
result = cc.loc[(cc['geoNetwork.region']!='N/A') & (cc['geoNetwork.city']=='N/A')]
result[result['geoNetwork.region'].duplicated(keep=False)]

,geoNetwork.city,geoNetwork.region,geoNetwork.country,0


In [383]:
result = cc.loc[(cc['geoNetwork.region']=='N/A') & (cc['geoNetwork.city']!='N/A')]
result[result['geoNetwork.city'].duplicated(keep=False)]

,geoNetwork.city,geoNetwork.region,geoNetwork.country,0


In [384]:
total.loc[train_index]['geoNetwork.country'].nunique(), total.loc[test_index]['geoNetwork.country'].nunique()

(222, 219)